In [ ]:
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import KerasTools as KT
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

import numpy as np
from keras.datasets import fashion_mnist
from keras import models
from keras import layers
from keras import optimizers
from keras.utils import to_categorical
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Load Fashion MNIST data and preprocess it
#
# Note: We will now use a 2D convolutional network, which expects input in format (height, width, channels)
# The MNIST data has only one channel per image (grayscale) (RGB images have 3 channels)
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [ ]:
# Build the neural network

def build_network():
    network = models.Sequential()
    # Convolutional base
    network.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    network.add(layers.MaxPooling2D((2, 2)))
    network.add(layers.Conv2D(64, (3, 3), activation='relu'))
    network.add(layers.MaxPooling2D((2, 2)))
    network.add(layers.Conv2D(64, (3, 3), activation='relu'))
    
    # DNN classifier
    network.add(layers.Flatten())
    network.add(layers.Dense(64, activation='relu'))
    network.add(layers.Dense(10, activation='softmax'))
    
    network.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

    return network

In [ ]:
network = build_network()
epochs = 5
history = network.fit(train_images, train_labels, epochs=epochs, batch_size=128)

test_loss, test_acc = network.evaluate(test_images, test_labels)
print()
print("Test loss", test_loss)
print("Test accuracy", test_acc)
history.history['test_loss'] = test_loss
history.history['test_acc'] = test_acc
history.history['epochs'] = epochs

In [ ]:
KT.plot_history(history.history)

In [ ]:
# Save network for later production use
network.save("./fmnist_convnet_trained.h5")

In [ ]:
entry = 0

test_image = test_images[entry:entry+1]
test_label = np.argmax(test_labels[entry])
desc = KT.datasets.decode.decode_dataset('fmnist', test_label)

pred = network.predict(test_image)
prediction = KT.datasets.decode.decode_predictions('fmnist', pred, top=10)

plt.title("FMNIST image no. {0} with label '{1}/{2}'".format(entry, test_label, desc))
plt.imshow(test_image.reshape(28,28), cmap=plt.cm.binary)
plt.show()

print('-' * 32)
print("{0:>6} | {1:>12} | {2:>6} ".format("Class", "Label", "Score"))
print('-' * 32)
for p in prediction[0]:
    print("{0:>6} | {1:>12} | {2:>7.2%} ".format(*p))
print('-' * 32)